<a href="https://colab.research.google.com/github/ShreyJ1729/EVA6-TSAI/blob/main/04-Backprop%26ArchBasics/04-Backprop%26ArchBasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [231]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # 28x28x1 --> 28x28x16 --> 26x26x16 --> 24x24x32 --> 12x12x32
        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),

            nn.Conv2d(16, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),

            nn.Conv2d(16, 32, 3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.05),
            nn.MaxPool2d(2, 2)
        )

        # 12x12x32 --> 12x12x16 --> 12x12x20 --> 6x6x20
        self.convblock2 = nn.Sequential(
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),

            nn.Conv2d(16, 20, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout2d(0.05),
            nn.MaxPool2d(2, 2)
        )
        
        # 6x6x20 --> 1x1x20
        self.convblock3 = nn.Sequential(
            nn.Conv2d(20, 20, 3, padding=1),
            nn.ReLU(), 
            nn.BatchNorm2d(20),
            nn.Dropout2d(0.05),
            nn.AvgPool2d(6)
        )

        self.fcblock = nn.Linear(20, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(-1, 20)
        x = self.fcblock(x)
        return F.log_softmax(x, dim=1)

In [232]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
         Dropout2d-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 26, 26]           2,320
              ReLU-6           [-1, 16, 26, 26]               0
       BatchNorm2d-7           [-1, 16, 26, 26]              32
         Dropout2d-8           [-1, 16, 26, 26]               0
            Conv2d-9           [-1, 32, 24, 24]           4,640
             ReLU-10           [-1, 32, 24, 24]               0
      BatchNorm2d-11           [-1, 32, 24, 24]              64
        Dropout2d-12           [-1, 32, 24, 24]               0
        MaxPool2d-13           [-1, 32, 12, 12]               0
           Conv2d-14           [-1, 16,

In [233]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [234]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        tqdm.write(f'\r[Epoch {epoch}] loss={round(loss.item(), 4)}', end="")


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    tqdm.write('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [235]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

[Epoch 1] loss=0.1433

Test set: Average loss: 0.1117, Accuracy: 9693/10000 (96.93%)



[Epoch 2] loss=0.061

Test set: Average loss: 0.0600, Accuracy: 9837/10000 (98.37%)



[Epoch 3] loss=0.1125

Test set: Average loss: 0.0428, Accuracy: 9866/10000 (98.66%)



[Epoch 4] loss=0.0711

Test set: Average loss: 0.0418, Accuracy: 9864/10000 (98.64%)



[Epoch 5] loss=0.0708

Test set: Average loss: 0.0329, Accuracy: 9896/10000 (98.96%)



...

[Epoch 15] loss=0.0185

Test set: Average loss: 0.0202, Accuracy: 9931/10000 (99.31%)



[Epoch 16] loss=0.0622

Test set: Average loss: 0.0197, Accuracy: 9928/10000 (99.28%)



[Epoch 17] loss=0.0144

Test set: Average loss: 0.0214, Accuracy: 9928/10000 (99.28%)



[Epoch 18] loss=0.0264

Test set: Average loss: 0.0189, Accuracy: 9940/10000 (99.40%)



[Epoch 19] loss=0.085

Test set: Average loss: 0.0200, Accuracy: 9927/10000 (99.27%)

